
# Memoization and Tabulation in Python 

"Dynamic Programming -  Learn to Solve Algorithmic Problems & Coding Challenges"
Available on YouTube:

[https://www.youtube.com/watch?v=oBt53YbR9Kk](https://www.youtube.com/watch?v=oBt53YbR9Kk)


(by Alvin from freeCodeCamp.org)


Ned Poland - November 2021

[https://www.linkedin.com/in/nate-poland/](https://www.linkedin.com/in/nate-poland/)

[https://github.com/np1919](https://github.com/np1919)

## Dynamic Programming

In the video above, Alvin clearly and elegantly breaks down the key concepts of dynamic programming in terms of solving complicated (algorithmic) problems at scale. This is my first work learning about **big-O notation, the time and space complexity of a computation**, and I thought it was a great introduction. 

My understanding is that **dynamic programming seeks to solve large problems by breaking them down into smaller ones that we know how to solve**.


In the video above, the code is written in Javascript, but I've written my implementations in Python. 

In the workbook below I attempt to solve a few (fairly simple) algorthmic problems, as described in the video. 

The functions are split broadly into two categories of algorithmic solving used in dynamic programming; 

# `memoization` (recursion) and `tabulation`

 

Memoization is the process of **storing the results of a function call for later reference**, to reduce the computation necessary to solve the algorithm as a whole.

This requires a 'memo' data container, to store the values you've already calculated -- and keep you from doing work that you've done before in the call stack. My first idea was to use a basic dictionary constructor as a default argument. 

However, due to way that Python functions store default arguments inside the function definition, caching a memo was more difficult than originally anticipated. 

Because it is the **function definition** which holds the default arguments, and not the **instance of the class**, only one dictionary was ever created -- and shared -- meaning that the call stack from one set of arguments was being used for all subsequent calls to the function. This is not the behaviour I was looking for.

I couldn't use @cache or @lru-cache, because some of the functions below return Python lists, which aren't hashable as they are mutable objects in Python. I wasn't really looking for a cache anyway -- it isn't necessary to summon the values up again; I simply wanted a fresh memo object for each call (without defining a new one each time -- which may in fact be a better option in many cases). 

---

If we define a default argument in the function definition, like so:

In [30]:
def can_sum(m:int, n:list, memo:dict=dict()):
    """ accepts:
                m: integer
                n: list of integers
        returns:
            bool;
             whether or not any combination of the integers in `n` can sum to the target, `m`.
            replacement is permitted. all integers in n will be non-negative.
        """
    #### Base Case/Win Condition; subtract successfully to 0.
    if m == 0: 
        return True

    #### Lose Condition; subtracted too far. Return False
    if m < 0:
        memo[m] = False
        return False
    
    #### Early Return --> target already in memory
    if m in memo:
        return memo[m]
    
    #### Branching/Recursive Logic;
            # Iterate through integers in `n`
    for element in n:
            # subtract `element` from m. Pass the remainder into a recursive call.
        remainder = m - element
            # calculate that branch/node;
        result = can_sum(remainder, n, memo)
        
            # if ever a node reaches 0, we can return True early. --> we Can Sum to the target. 
        if  result == True:
            memo[m] = True
            return True
        else:
            # if not, return False. Memo-ize in either case. As long as a recursive call is still running from this branch, 
            # the value at this stage of `m` will not be set to False in the dictionary
            memo[m] = False
            return False

Then only one dictionary is ever created, and it sits in the function definition:

In [32]:
can_sum

<function __main__.can_sum(m: int, n: list, memo: dict = {})>

When we call the function without assigning a new memo object:

In [34]:
can_sum(8, [1,2,3,5])

True

The memo values for the function called with these arguments are then stored in the **default arguments of the function definition**

In [39]:
can_sum

<function __main__.can_sum(m: int, n: list, memo: dict = {1: True, 2: True, 3: True, 4: True, 5: True, 6: True, 7: True, 8: True})>

If we then call the function with different arguments (again, without entering our own memo object):

In [40]:
can_sum(11, [4, 8, 14])


True

You end up with an incorrect answer. Unfortunately, you need to pass a new memo object to each function call, even if passing the dict() call into each function looks clunky.

Which gives erroneous results when we call the function with new parameters:

In [45]:
print(can_sum(8, [2,3,5], dict()))
print(can_sum(11, [4, 8, 14], dict()))
print(can_sum(3000, [7,14], dict())) # note that this SHOULD return false, but because the memo was stored in the function dictionary,
                            # it returns true. Below, you can also see the many values which are not multiples of 7 (or 14)
                            # which are now stored in the function's (bound?) `memo` dictionary
# can_sum

# returns:
# True
# False
# False
# ... as expected.

True
False
False


If we redefined the function, and ran the cells in opposite order, we would return False for both

---

# Recursion

## Introduction

In recursive algorithms, we break a problem down into smaller and smaller parts until we reach a **Base Case** scenario -- which either produces our desired result, or proves it's impossibility.

For example, if our desired result was that a target integer was to be reduced to 0 by *subtracting* (positive) integers from another list --> the 'base case' would be that the result of our function call ends up making 0. Conversely, the impossible 'base case' would be if our function call returns any number less than 0 --> since we're subtracting positive integers, we can't add to the final value in any way from there.

If our goal was to reach 0, we can eventually reach 0 by performing the operation `-1` on the value 1. We can continue on in that way until we've subtracted the whole integer, if we are allowed to use at least 10 `-1`'s.

In [46]:
def sub_1(m):
    ''' subtract 1 from m until you reach 0'''
    
    # Base Case; 
    # m reaches 0.
    if m == 0:
        return True
    if m > 0:
        # verbose
        print(m, end='\t')       
        # call the function inside itself (recursively). 
        return sub_1(m-1)
        
sub_1(10)
# despite the other calls needing to resolve before 
# calculating the value of sub_1(10); 
# the results are displayed in order.

10	9	8	7	6	5	4	3	2	1	

True

In the above, there's no need to use recursion -- subtracting a known integer from another known integer takes O(1) time (I think?), whereas we are performing 10 operations instead -- that's bad. Another way to criticise would be to say that there's only one 'branch' to the tree -- because we can only subtract 1 -- which means a recursive algorithm was definitely not necessary.

The problems below add more variables to that second (now set of) value(s); such that we have an array of values, `n`, which we use to achieve some desired result with relation to a target, `m`. These values could be sequences(strings), or integers, etc., depending on the use case.

---

## Memoization for Recursive Problems

**Memoization** is the idea that the smallest iteration of a dynamic programming problem will likely have to be calculated many many times. It seeks to store the values for those simple calculations in a record; from there, it can simply reference the record when the problem comes up again, instead of having to re-calculate the value.

Problems are often visualized as 'recursive trees' in this method, where, for example; from a large sequence (head node), we begin to chip away at the values, breaking them into smaller and smaller chunks by means of a process we can eventually perform in reverse. If we are able to fully *digest* the problem using our very small method --> then we can reconstruct our steps, and are sure we can climb back up to the top of the tree (thereby solving our big problem). 

Typically, many of the branches will not satisfy our needs. In some cases we only need to find a single path back, and for other problems we might want to find all paths. Within these requirements we hope to keep our **time and space complexity** to a minimum; *linear is much better than exponential, especially at scale.*


In memoization problems, we use a cached memory to save the results from our (smaller) recursive calls in memory. By doing so, we reduce the number of calculations (*time*) necessary to complete the problem. The amount of *space* necessary does remain relevant in this type of solution.

---

## Grid Traveler

The goal in this case is to move from the top left of an 18 * 18 grid to the bottom right, and count how many different ways you could get there. 

The **Base Case** here is that you end up at position (1,1) -> able to move off the last corner of the grid. 


If we imagine the grid instead as x and y cartesian points, we could say that any positions where the x or y point is at 0 is **out of bounds**; equally, any x or y value greater than the (square) grid would be out of bounds. Therefore; starting at the top left of an 18\*18 grid, how many ways are there to find yourself at the bottom right, moving only right or down?

There will be only one path to the finish once the traveler reaches x=1 and y=1. This means you're at the corner, and moving right or down (the only legal moves) will result in success.

Note: Grid Traveler can use the same cache, as you are always moving by 1 and the board follows linear paths. 

In [51]:
def grid_traveler(m:int, n:int, memo:dict=dict()):
    '''beginning at the top left of a 2-D grid, 
        and moving only DOWN or to the RIGHT;
         HOW MANY WAYS can you travel to the goal?'''

    # Base Cases;
    if m == 1 and n == 1:# Win -> you've reached the bottom right corner.
        return 1
    
    if m == 0 or n == 0: # Lose -> you're out of bounds.
        return 0

     
    # Memoization
    if (m, n) in memo: 
        return memo[(m, n)]
    if (n, m) in memo:
        return memo[(n, m)]

    
    go_down = grid_traveler(m-1, n, memo) # move down
    go_right = grid_traveler(m, n-1, memo) # move right
    
    total = go_down + go_right # the total will be the sum of the potential paths stemming from this node
    memo[(m, n)] = total # since the underlying grid will never change, keep cache in func.

    return total
        


# Wrapping this function means calculating each value again for each call -- 
# This is redundant, as the total # of paths for each 'node' will remain constant
# Over any number of larger grids. 
# 
print('lose cases')
print(grid_traveler(0, 0))
print(grid_traveler(18,0))
print(grid_traveler(0,18))
print('how many paths to the exit?')
print(grid_traveler(2,3))
print(grid_traveler(3,3))
print('base win conditions')
print(grid_traveler(1,1))
print(grid_traveler(18,1))
print(grid_traveler(1,18))
print('for an 18x18 grid;')
print(grid_traveler(18,18))
# grid_traveler

lose cases
0
0
0
how many paths to the exit?
3
6
base win conditions
1
1
1
for an 18x18 grid;
2333606220


In [85]:
from collections import defaultdict
counter = defaultdict(int)

for x, y in grid_traveler.__defaults__[0].keys():
    counter[x] += 1
    counter[y] += 1
len(grid_traveler.__defaults__[0])

170

In [86]:
counter

defaultdict(int,
            {1: 17,
             2: 19,
             3: 19,
             4: 19,
             5: 19,
             6: 19,
             7: 19,
             8: 19,
             9: 19,
             10: 19,
             11: 19,
             12: 19,
             13: 19,
             14: 19,
             15: 19,
             16: 19,
             17: 19,
             18: 19})

how_sum
---

Find *any way* to create the target integer `m` using the elements in `n`, with replacement.

We can return early as soon as we find a complete path. 


In [46]:

def how_sum(m:int, n:list, memo=dict()):
    '''return one possible combination of sub-arrays in `n` which sum to produce `m` '''
    # Base case; you've removed a sub-array from m and m is now 0; you win.
    if m == 0:
        return []
    # Base failure case; you've removed a sub-array from m and m is now less than 0; you lose.
    if m < 0:
        return None
    
    # If you've done this before, don't.
    if m in memo:
        return memo[m]
    
    # iterate through elements in n.
    for num in n:
        remainder = m - num # create all possible remainders.
        chain = how_sum(remainder, n, memo) # for each remainder, create a chain recursive call.
        if chain is not None:            # when one eventually resolves, concatenate with this node and pass upwards
            result = chain + [num]       #
            memo[m] = result             # store the result and return it
            return result
        
        else:                           # if the recursive call finishes and has no hits, this node is also a dud.
            memo[m] = None
    
    
print(how_sum(7, [2,3]))
print(how_sum(7, [5,4])) # INCORRECT returned list.
print(wrapper(how_sum,8, [2,3,5])) # this is correct, but the longest possible combination...
print()
print(wrapper(how_sum,7, [4,6]))
print(wrapper(how_sum,300, [7, 14])) # from inside the wrapper, these calls correctly return None.

[3, 2, 2]
[3, 2, 2]
[2, 2, 2, 2]

None
None


best_sum
---
Find the shortest combination of elements in `n` which sum to target integer `m`.

We'll have to exhaust every combination of our tree and examine each possible outcome. 

In [47]:
def best_sum(m:int, n:list, memo=dict()):
    """accept:
        m: int -> the target sum
        n: list: -> a list of integers with which to create the sum
        memo: dict -> defaults to an empty dictionary. To save results simply
        pass another dictionary.
        
        returns:
             the shortest possible array of numbers from `n` which sum to `m`
     
     """
    # Base Case: Win; you've called recursively with an m value of 0.
    if m == 0:
        return []
    # Base Case: Lose; you've gone too far.
    if m < 0:
        return None
    if m in memo:
        return memo[m]
    
    shortest = None # could easily be adapted to also provide the Longest combination
    
    for num in n:
        remainder = m - num
        chain = best_sum(remainder, n, memo) # returns a list, or None

        if chain is not None:
            combination = chain + [num] # chain returns as the shortest combination of the recursive call below

            if shortest == None: # first time through
                shortest = combination
            if len(combination) < len(shortest): # check that this combination is the shortest one.
                shortest = combination
         
            memo[m] = shortest # store in memory
    
    return shortest # return the shortest combination. 


## Example using wrapper
print(wrapper(best_sum, 10, [1,2,3,4,5]))
print(wrapper(best_sum, 7, [3,4,5,7]))
print(wrapper(best_sum, 8, [1,4,5]))
print(wrapper(best_sum, 100, [1,5,25]))
print(wrapper(best_sum, 100, [1,5,25,100])) # notice that this 100 is not saved.
print(wrapper(best_sum, 1000, [17,55,1,5,25]))

[5, 5]
[7]
[4, 4]
[25, 25, 25, 25]
[100]
[5, 5, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55]


can_construct
---

Indicate whether or not a target *string* `m` can be created using the (sub-)sequences in `n`, with replacement. 

Return a bool.

In [48]:
def can_construct(m:str, n:list, memo=dict()):
    """ define a function can_construct:
    accepts:
        `m` : a sequence
        `n` : list of (sub)sequences, which can be used with replacement
        optional: a memo-ization object, or cache; to store the results of our recursive calls. Defaults to an empty dictionary.
    returns:
        boolean value; whether the m, `m`, can be created from the elements of list, `n`
        """
        #### Base Case/Win Condition --> m sequence `m` goes to 0.
            #### Return True.
    if m == "":
        memo[m] = True
        return True
    
        #### Early Return --> Already in Memo
    if m in memo:
        return memo[m]
    
        #### Branch Logic;
        #### Iterate through elements of sequence `n` and look for a prefix (match) with the current `m` value. Subtract it
        #### and keeping going

    for word in n:
        if m.startswith(word): # if word is a prefix of m;
            chain = can_construct(m[len(word):], n) # call recursively on the remainder of the list
            if chain == True: # Return early on the first True
                memo[m] = True 
                return True

        # If the whole chain isn't true by the end of the loops;
        # save this `m` value in memory as False and return False
    memo[m] = False
    return False



print(wrapper(can_construct, 'abcdef', ['ab', 'abc', 'cd', 'def', 'abcd', 'f']))
print(wrapper(can_construct, 'skateboard', ['bo', 'rd', 'ate','t','ska','sk','boar']))
print(wrapper(can_construct, 'enterapotentpot', ['a','p','ent','enter','ot','o','t']))
print(wrapper(can_construct, 'eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeef', ['e','ee','eee','eeee','eeeee','eeeeee']))


True
False
True
False


allConstruct
---
Return **all possible** combinations of the elements in `n` which can create the sequence `m`.

We'll have to calculate the whole tree to check all combinations.

In [49]:
def all_construct(m, n):
    """ 
    accepts:
        m: a string
        n: a list of (sub-)strings, which can be used with replacement

    returns:
        2-D array of all combinations of n which produce m
        """
        # Base Case; you've removed elements and found str(len(0)). You win.
    if m == "":
        return [[]]
    
        # container to hold all possible combinations
    results = []
    for word in n:
        if m.startswith(word): # for this 'suffix', `m`;
            results += [[word] + x for x in all_construct(m[len(word):], n)] 
                        # prepend word to each inner list of the list returned by the recursive call
            # for each nested list returned from the recursive call, add this word to that list and pass 
            # the full list of combinations from this node upwards.

        # return the container of all possible combinations from this node's branches and leaves. 
    return results


print(all_construct('purple', ['purp', 'le', 'p', 'urple', 'e']))

print(all_construct('abcdef', ['ab', 'abc', 'cd', 'def', 'abcd', 'ef']))
print(all_construct('skateboard', ['bo', 'rd', 'ate','t','ska','sk','boar']))
print(all_construct('enterapotentpot', ['a','p','ent','enter','ot','o','t']))
print(all_construct('eeeeeeeeeeeeeeeeeeeeeef', ['e','ee','eee','eeee','eeeee','eeeeee']))


[['purp', 'le'], ['p', 'urple']]
[['ab', 'cd', 'ef'], ['abc', 'def'], ['abcd', 'ef']]
[]
[['enter', 'a', 'p', 'ot', 'ent', 'p', 'ot'], ['enter', 'a', 'p', 'ot', 'ent', 'p', 'o', 't'], ['enter', 'a', 'p', 'o', 't', 'ent', 'p', 'ot'], ['enter', 'a', 'p', 'o', 't', 'ent', 'p', 'o', 't']]


[]


---

# Tabulation

Tabulation involves using a table to create an ordered, progressive solution for a problem.

Alvin's rules for tabulation are as below:

    -visualize the problem as a table
    -size the table based on the inputs (usually length of target, m)
    -initialize the table with default values
    -seed the trivial answer into the table
    -fill further positions based on the current position
    
For these purposes, I'll bring in pandas and numpy :)

In [50]:
import pandas as pd
import numpy as np

Fibonacci
---


In [51]:
def fib(n):
    current, previous = 0, 1
    for x in range(n):
        current, previous = current + previous, current # quick and easy tuple unpacking.
    return current
print(fib(5))
print(fib(55))
print(fib(555))

5
139583862445
43516638122555047989641805373140394725407202037260729735885664398655775748034950972577909265605502785297675867877570


tab_grid_traveler
---

In [52]:
def tab_grid_traveler(m, n):
    '''accept:
            cartesian coordinates (m, n)
            
        return:
            number of paths to (1,1)
    '''
    mat = np.zeros((m+1, n+1)) # form the grid
    table = pd.DataFrame(mat)
    start = table.iloc[1,1] = 1 # seed the table
    for rowidx, row in table.iterrows(): 
        for colidx, col in enumerate(row): # for each position, moving from top left;
            try:
                table.iloc[rowidx+1, colidx] += table.iloc[rowidx, colidx] # add the cell values to the one below
            except:
                pass        # naive exception handling deals with moving out-of-bounds
            try:
                table.iloc[rowidx, colidx+1] += table.iloc[rowidx, colidx] # add the cell values to the one to the right
            except:
                pass
   
    
    return table.iloc[-1,-1] # can be easily modified to return the actual df; currently returns the count at (1,1)


display(tab_grid_traveler(3,3))
display(tab_grid_traveler(6,6))
tab_grid_traveler(18,18)

6.0

252.0

2333606220.0

tab_can_sum
---


In [53]:
def tab_can_sum(m, n):
    '''
    accept:
    m; integer
    n; list of integers
    
    return:
     ; bool
         whether or not the target sum, `m`, can be constructed using the elements of `n`
         with replacement
     '''
    arr = pd.Series(False, index=range(m+1)) # starting at base case 0 (index 0);
    arr[0] = True # seed True value at m==0;
    for idx, val in enumerate(arr): 
        if val == True: # if the cell has been seeded;
            for num in n: 
                if not idx+num > m: # if the resulting index is not out of bounds;
                    arr[idx+num] = True # seed the cell
    
    return  arr.index[m], arr[m]

print(tab_can_sum(7, [5,3,4]))
print(tab_can_sum(300, [7,14]))

(7, True)
(300, False)


tab_how_sum
---

In [54]:
def tab_how_sum(m, n):
    '''
          stores only one value in each cell.
        iterates backwards over the list to 'pick up' the stored values.
    
    accepts:
            m: integer
            n: list of integers
        returns:
            one possible combination of elements from `n`, with replacement, which sum to produce `m`.
            
      
    '''
    

    # create table
    arr = [None for x in range(m+1)] # we're summing to a number, index m+1
    arr[0] = [] # seed the table. this is our winning 'base case'.
    
        # container for return values
    output = []
    

    for idx, val in enumerate(arr):
        if val is not None: # if you're at a proven viable position; 
            for num in n: # for each element in `n`.
                if idx+num < len(arr): # limit so you won't move out of bounds;
                    
                    if arr[idx+num] is not None:
                        if num > arr[idx+num]: ####### check if the stored value is the largest... 
                                                ############ this isn't Necessarily the best route, but it is the biggest step.
                                
                            arr[idx+num] = num # assign the number that got you there to the resulting index.
                    else:
                        arr[idx+num] = num # assign the number that got you there to the resulting index.

                    
                    
            # then simply iterate backwards and pick up your numbers.
    if arr[m] is not None:
        position = m
        while position != 0:
            output.append(arr[position])
            position = position - arr[position] 
    return output


print(tab_how_sum(7, [1,2,3]))
print(tab_how_sum(8, [2,3,5]))
print(tab_how_sum(7, [5,3,4, 7]))
print(tab_how_sum(300, [7,14])) # Returns an empty list instead of None
print(tab_how_sum(100, [1,2,5,25]))
print(tab_how_sum(100, [1,2,5,25,31])) # note that this does not correctly identify [25,25,25,25] as the best solution



[3, 3, 1]
[5, 3]
[7]
[]
[25, 25, 25, 25]
[31, 31, 31, 5, 2]


In [55]:
def tab_how_sum2(m, n):
    ''''version two!
         instead of iterating backwards to 'pick up' the elements, it concatenates them in the forward iteration
        (no chance to filter)    
        
    accepts:
            m: integer
            n: list of integers
        returns:
            one possible combination of elements from `n`, with replacement, which sum to produce `m`.
            
               
    '''
    
    arr = [None for x in range(m+1)] # we're summing to a number, index m+1M
    arr[0] = [] # seed the table

    for idx, val in enumerate(arr): 
        if val is not None: 
            for num in n:
                if idx+num < len(arr):
                    if arr[idx+num] == None:
                        arr[idx+num] = val + [num]  
                    else:  
                        arr[idx+num] = val + [num]
                        
                        
    return arr[m]

print(tab_how_sum2(7, [2,3]))
print(tab_how_sum2(8, [2,3,5]))
print(tab_how_sum2(7, [5,3,4, 7]))
print(tab_how_sum2(300, [7,14])) # Returns None instead of an empty list
print(tab_how_sum2(100, [1,2,5,25]))
print(tab_how_sum2(100, [1,2,5,25,31])) # this solution is much more naive



[3, 2, 2]
[2, 2, 2, 2]
[4, 3]
None
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


tab_best_sum
---

Synthesis of the two answers to tab_how_sum, above.

In [56]:
def tab_best_sum(m, n):
    ''' accepts:
            m: an integer
            n: a list of integers
    
    returns:
        the SHORTEST combination of elements of `n` which sum to `m`
    '''
    output = []
    arr = [None for x in range(m+1)] 
    arr[0] = [] 

    for idx, val in enumerate(arr): 
        if val is not None:
            for num in n:
                if idx+num < len(arr):
                    if arr[idx+num] == None:
                        arr[idx+num] = val + [num]  
                    elif len(val + [num]) < len(arr[idx+num]):#### Add condition 
                                                              #### (that the replacement is shorter)
                        arr[idx+num] = val + [num]

    return arr[m]


print(tab_best_sum(7, [2,3]))
print(tab_best_sum(8, [2,3,5]))
print(tab_best_sum(7, [5,3,4, 7]))
print(tab_best_sum(100, [1,2,5,25]))
print(tab_best_sum(100, [1,2,5,25,31])) # this solution correctly identifies the length of each substep 
                                        # (intead of just the value of its most recent step, as in tab_how_sum)

[2, 2, 3]
[3, 5]
[7]
[25, 25, 25, 25]
[25, 25, 25, 25]


tab_can_construct
---

In [57]:
import pandas as pd
def tab_can_construct(m, n):
    '''accept:
            m: a string
            n: a list of strings
            
        return:
            BOOL 
            
            whether or not the target string`m`
            can be constructed 
            using the elements from `n `
            with replacement'''
 

    # create table of length target string + 1
    arr = pd.Series(index=[x for x in m] + ['Final'], data = False)
    arr[0] = True # seed the viable case
    
    for idx, (key, val) in zip(range(len(m)+1), arr.iteritems()): # for each possible starting position
        if arr[idx] == True: # if it has been proven viable (we move right-wise)
            current_letter = arr.index[idx]            
            for sub in n: # check each subsequence in n   
                if sub.startswith(current_letter) and (idx+len(sub) < len(arr)):
                    arr[idx+len(sub)] = True
    return arr[-1]

tab_can_construct('abcdef', ['ab', 'abc', 'cd', 'def', 'abcd'])

C:\Users\polan\AppData\Local\Temp\ipykernel_3384\448419389.py:18: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  arr[0] = True # seed the viable case


AttributeError: 'Series' object has no attribute 'iteritems'

tab_count_construct
---


In [ ]:
def tab_count_construct(m, n):
    '''accept:
            m: a string
            n: a list of strings
            
        return:
                INT
                
            HOW MANY WAYS `m`can be constructed 
            using the elements from `n `
            with replacement'''
 

    # create table of length target string + 1
    arr = pd.Series(index=[x for x in m] + ['Final'], data = 0)
    arr[0] = 1 # seed the viable case
    
    for idx, (key, val) in zip(range(len(m)+1), arr.iteritems()): # for each possible starting position
        if arr[idx] > 0: # if it has been proven viable (we move right-wise)
            for sub in n:
                    # determine potential next index, based on subsequences in n
                next_index = idx+len(sub)
                    # check each subsequence in n against the suffix (m[idx:next_index]) 
                    # if the substring matches; and the next_index is valid;
                if sub == m[idx:next_index] and (next_index < len(arr)): #len is indexed from 1
                    arr[idx+len(sub)] += arr[idx] 
                        # add the paths to the next index
    return arr[-1]

print(tab_count_construct('abcdef', ['ab', 'abc', 'cd', 'def', 'abcd']))
print(tab_count_construct('eeeeeeeeeeeeeeeeef', ['ef', 'e', 'eeeeee','eee','eeee',]))
print(tab_count_construct('eeeeeeeeeeeeeeeeef', ['eeeeeeeeeef', 'e', 'eeeeee','eee','eeee',]))

tab_all_construct
---

Create a table containing all possible ways to construct target string `m` from the sub-sequences in `n`, with replacement.

In [ ]:
def tab_all_construct(m, n):
    '''accept:
            m: a string
            n: a list of strings
            
        return:
            2-D array of ALL WAYS `m`can be constructed 
            using the elements from `n `
            with replacement'''
    n = set(n)
    m_string = [x for x in m] + ['Final'] # m+1
    arr = [None for x in m_string] # matching list
    arr[0] = [[]] # seed the viable case

    for idx, char in enumerate(m_string):
        suffix = m[idx:]
        if isinstance(arr[idx], list): # a list; a viable path.
            for sub in n: # check each subsequence in n
                if suffix.startswith(sub):
                    next_index = idx+len(sub)
                    # everything in arr[idx] -> next_idx
                    if arr[next_index] == None:    
                        arr[next_index] = [x+[sub] for x in arr[idx]]
                    else:
                        arr[next_index] += [x+[sub] for x in arr[idx]]

    return pd.Series(arr, index=m_string)[-1]



print(tab_all_construct('abcdef', ['ab', 'abc', 'cd', 'def', 'abcd']))
print(tab_all_construct('panepaneeeee', ['p','an','e']))

print(len(tab_all_construct('eeeeeeeeeeeeeeeeef', ['ef', 'e', 'eeeeee','eee','eeee',])))
print(len(tab_all_construct('eeeeeeeeeeeeeeeeef', ['eeeeeeeeeef', 'e', 'eeeeee','eee','eeee',])))

These last three have all been coded to return only the last value, but the full tables/Series are available inside the function. 

I hope you've enjoyed reading this notebook about `memoization` and `tabulation` in Python!